In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import linear_kernel
import requests
from io import StringIO


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances


In [2]:
# Load the data
movies = pd.read_csv('/Users/sakshi/Desktop/Applied_AI/ML2/ml-20m/movies.csv')
ratings = pd.read_csv('/Users/sakshi/Desktop/Applied_AI/ML2/ml-20m/ratings.csv')
tags = pd.read_csv('/Users/sakshi/Desktop/Applied_AI/ML2/ml-20m/tags.csv')


In [3]:
# Combine genres into a single string
movies['genres'] = movies['genres'].fillna('')
movies['content'] = movies['title'] + ' ' + movies['genres']


In [4]:

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['content'])

In [5]:
# Dimensionality reduction using Truncated SVD
svd = TruncatedSVD(n_components=50, random_state=42)
latent_matrix_1 = svd.fit_transform(tfidf_matrix)


In [6]:
# Standardize the latent matrix
scaler = StandardScaler()
latent_matrix_1_standardized = scaler.fit_transform(latent_matrix_1)

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix

import pandas as pd
from scipy.sparse import csr_matrix

# Assuming 'ratings' is a pandas DataFrame
# Create a sparse user-movie matrix
user_movie_matrix = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

# Note: This assumes that 'userId' and 'movieId' are integer indices. Adjust accordingly.

# Now, 'user_movie_matrix' is a sparse matrix representing the user-movie ratings.
# You can perform various operations on this sparse matrix without fully materializing it.

# For example, to get the rating for a specific user and movie:
user_id = 1
movie_id = 123
rating = user_movie_matrix[user_id, movie_id]

# To get all ratings for a specific user:
user_ratings = user_movie_matrix[user_id, :]

# To get all ratings for a specific movie:
movie_ratings = user_movie_matrix[:, movie_id]

/var/folders/r6/9zvg424x00l_fnkr5bl4pmjm0000gn/T/ipykernel_7615/3998833273.py:2: PerformanceWarning: The following operation may generate 3703856792 cells in the resulting pandas object.
  user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)


: 

In [46]:












# Normalize by subtracting mean
user_movie_matrix_mean = user_movie_matrix.mean(axis=1)
user_movie_matrix_normalized = user_movie_matrix.sub(user_movie_matrix_mean, axis=0)

# Fill NaN values with 0
user_movie_matrix_normalized = user_movie_matrix_normalized.fillna(0)

# Calculate user similarity using cosine similarity
user_similarity = cosine_similarity(user_movie_matrix_normalized)

# Hybrid Model

# Combine content-based and collaborative filtering matrices
hybrid_matrix = np.concatenate([latent_matrix_1_standardized, user_similarity.T], axis=1)

# User-Item Rating Matrix for Evaluation
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Train-test split for evaluation
train_matrix, test_matrix = train_test_split(user_item_matrix.values, test_size=0.2, random_state=42)

# Predict using hybrid model
predictions = pairwise_distances(hybrid_matrix, metric='cosine').dot(train_matrix) / np.abs(pairwise_distances(hybrid_matrix, metric='cosine').sum(axis=1, keepdims=True))

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(test_matrix[test_matrix.nonzero()], predictions[test_matrix.nonzero()])
print(f'Mean Squared Error: {mse}')

/var/folders/r6/9zvg424x00l_fnkr5bl4pmjm0000gn/T/ipykernel_4316/1314788037.py:20: PerformanceWarning: The following operation may generate 3703856792 cells in the resulting pandas object.
  user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x104b75300>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tempfile.py", line 449, in __del__
    def __del__(self):

KeyboardInterrupt: 


KeyboardInterrupt: 